# Librerías necesarias

In [19]:
import os
os.chdir('../movies')
from movieLens import MovieLens

import numpy as np
import itertools
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

from collections import defaultdict

# Lectura de ficheros

In [2]:
ml = MovieLens()

In [3]:
ratings = ml.ratings
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
# Define the Reader object to parse the dataframe
reader = Reader(rating_scale=(ratings['rating'].min(), ratings['rating'].max()))

# Load the dataframe as a ratings dataset
ratingsDataset = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

#trainset = ratingsDataset.build_full_trainset()

# Entrenamiento del modelo

In [5]:
# Hago este split para sacar métricas después
trainset, testset = train_test_split(ratingsDataset, test_size=0.2, random_state=42)

In [6]:
# Voy a usar la misma configuración y el mismo modelo que has usado en tu caso
sim_options = {'name': 'cosine', 'user_based': True}
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)
simsMatrix = model.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


# Comparación de similitudes

Con respecto al punto que hablábamos el otro día sobre que los usuarios se parecen demasiado unos con otros y dan exactamente uno. He indagado un poco (**bastante**) en el código de surprise y he visto como se realiza el cálculo de la similitud, te lo replico exactamente para que veas como se calcula.

Para este ejemplo, voy a comparar el usuario 11 con el usuario 1. El primer paso es comprobar que tienen películas en común, para ello hago una consulta sobre dos datasets filtrados. El primero corresponde a las valoraciones que ha hecho el usuario 1 a todas las películas que han visto tanto él como el usuario 11.

Además, voy a añadir un par de columnas de interés:

* rating_adj := rating - media de valoraciones que ha realizado el usuario 1 sobre todas las películas hasta ahora
* rating_sqr := rating * rating

Para el cálculo que hace surprise solo hará falta el rating y el rating_sqr pero más abajo te dejaré una alternativa para que cojas otras ideas. Esa nueva idea también se explica en el curso de Udemy (lección 35).

In [7]:
user1 = ratings[(ratings["userId"] == 1) & (ratings["movieId"].isin(ratings[ratings["userId"] == 11]["movieId"].values.tolist()))].copy()
mean1 = ratings[ratings["userId"] == 1]["rating"].mean()
user1["rating_adj"] = user1["rating"] - mean1
#user1["rating_sqr"] = user1["rating_adj"] * user1["rating_adj"]
user1["rating_sqr"] = user1["rating"] * user1["rating"]
user1

,userId,movieId,rating,timestamp,rating_adj,rating_sqr
2,1,6,4.0,964982224,-0.366379,16.0
7,1,110,4.0,964982176,-0.366379,16.0
19,1,349,4.0,964982563,-0.366379,16.0
20,1,356,4.0,964980962,-0.366379,16.0
25,1,457,5.0,964981909,0.633621,25.0
26,1,480,4.0,964982346,-0.366379,16.0
34,1,593,4.0,964983793,-0.366379,16.0
37,1,648,3.0,964982563,-1.366379,9.0
40,1,733,4.0,964982400,-0.366379,16.0
41,1,736,3.0,964982653,-1.366379,9.0


De la misma manera, pongo las valoraciones que ha hecho el usuario 11 sobre las películas que ambos han visto previamente con las columnas que he mencionado anteriormente.

In [8]:
user11 = ratings[(ratings["userId"] == 11) & (ratings["movieId"].isin(ratings[ratings["userId"] == 1]["movieId"].values.tolist()))].copy()
mean11 = ratings[ratings["userId"] == 11]["rating"].mean()
user11["rating_adj"] = user11["rating"] - mean11
#user11["rating_sqr"] = user11["rating_adj"] * user11["rating_adj"]
user11["rating_sqr"] = user11["rating"] * user11["rating"]
user11

,userId,movieId,rating,timestamp,rating_adj,rating_sqr
1259,11,6,5.0,902154266,1.21875,25.0
1264,11,110,5.0,902154266,1.21875,25.0
1272,11,349,5.0,902154342,1.21875,25.0
1273,11,356,5.0,901200263,1.21875,25.0
1279,11,457,5.0,902154316,1.21875,25.0
1282,11,480,4.0,902154383,0.21875,16.0
1287,11,593,5.0,902155102,1.21875,25.0
1288,11,648,4.0,902154514,0.21875,16.0
1289,11,733,4.0,902154431,0.21875,16.0
1290,11,736,4.0,902154542,0.21875,16.0


In [1]:
%%latex
La fórmula que Surprise usa es la siguiente:
Consideramos dos usuarios $u$ y $v$ y el conjunto de peliculas en común $I_{u,v}$, la similitud coseno entre esos dos usuarios es:
\begin{equation}
CosSim(u,v) = \dfrac{\sum_{p \in I_{u,v}} r_{u,p} . r_{v,p}}{\sqrt{\sum_{p \in I_{u,v}} r_{u,p}^{2}}.\sqrt{\sum_{p \in I_{u,v}} r_{v,p}^{2}}}
\end{equation}
Donde $r_{u,p}$ y $r_{v,p}$ son las valoraciones que han hecho el usuario $u$ y el usuario $v$ a la película $p$ respectivamente.

<IPython.core.display.Latex object>

**IMPORTANTE: Dos usuarios tendrán similitud de coseno = 1 si solamente coinciden en 1 película. Independientemente de las puntuaciones de las mismas**

In [10]:
# Traduciendo esa fórmula a lo que tenemos antes nos queda lo siguiente
np.dot(user1["rating"],user11["rating"])/((np.sqrt(user1["rating_sqr"].sum()*user11["rating_sqr"].sum())))

0.9751728395878438

Ahora nos falta saber si este valor se corresponde "más o menos" con el valor que devuelve la matriz de similitud en la fila correspondiente

In [11]:
trainset.to_inner_uid(1)

92

In [12]:
simsMatrix[trainset.to_inner_uid(1)]

array([0.9790026 , 0.95639518, 0.96523686, 0.95619822, 0.90796726,
       0.91547623, 0.98321417, 0.84147045, 0.97935342, 0.99398401,
       0.89593552, 0.95436193, 0.97289386, 0.95255397, 0.96129144,
       0.97273477, 0.97451397, 0.98900715, 0.97862465, 0.95022147,
       0.97196892, 0.97799673, 0.95073181, 0.99021457, 0.93973495,
       0.98245894, 0.97218008, 0.95253043, 0.95786433, 0.96707847,
       0.96236654, 0.88143721, 0.96770597, 0.97110047, 0.89983063,
       0.98497158, 0.97431329, 0.9761704 , 0.98045888, 0.9672222 ,
       0.97545348, 0.95347343, 0.95292578, 0.96926384, 0.97133158,
       0.93860308, 0.96755889, 0.95720003, 0.98458232, 0.97203627,
       0.9757757 , 0.95049533, 0.98275862, 0.98941927, 1.        ,
       0.98445946, 0.97743735, 0.9310134 , 0.99331562, 0.96510057,
       0.95131242, 0.9702471 , 0.96167521, 0.94471646, 0.98951352,
       0.93227453, 0.97100397, 0.97391304, 0.96875118, 0.96102027,
       0.97854533, 0.96192811, 0.88821764, 0.90380398, 0.99142

In [13]:
# Antes voy a verificar que la posición 92,92 es un 1 ya que un usuario está siempre relacionado consigo mismo.
simsMatrix[trainset.to_inner_uid(1)][trainset.to_inner_uid(1)]

1.0

In [14]:
# Ahora verifico que el valor de similitud entre los usuarios 1 y 11 es el mismo
simsMatrix[trainset.to_inner_uid(1)][trainset.to_inner_uid(11)]

0.9709753924554708

In [15]:
# Identicamente, ya que esta matriz debe ser simétrica, tenemos:
simsMatrix[trainset.to_inner_uid(11)][trainset.to_inner_uid(1)]

0.9709753924554708

Hay que tener en cuenta que para el modelo he usado el 80% de los datos, mientras que para la comprobación he usado todos, de ahí la diferencia de 0.005 entre las dos métricas de similitud. Aún así, he probado con todo el dataset y me sale exactamente igual. 

**Revisa tu notebook porque a lo mejor los usuarios que te dan 1 no son exactamente ese número por la identificación interna que hace el modelo**

In [113]:
%%latex
Alternativamente y teniendo en cuenta el aspecto que mencionabas el otro día sobre los diferentes baremos que hay entre usuarios se propone la siguiente formula de similitud alternativa:
\begin{equation}
CosSim(u,v) = \dfrac{\sum_{p \in I_{u,v}}(r_{u,p} - \hat{r_{u}}).(r_{v,p} - \hat{r_{v}})}{\sqrt{\sum_{p \in I_{u,v}} (r_{u,p} - \hat{r_{u}})^{2}}.\sqrt{\sum_{p \in I_{u,v}}(r_{v,p}-\hat{r_{v}})^{2}}}
\end{equation}
Donde $\hat{r_{u}}$ y $\hat{r_{v}}$ son las valoraciones medias que han hecho el usuario $u$ y el usuario $v$ en cada una de las películas que han valorado respectivamente (independientemente de que sean comunes entre ellas o no).

<IPython.core.display.Latex object>

Con esta versión alternativa se intenta paliar el efecto de puntuar de manera similar con baremos de medida diferente. Los datasets de arriba están casi preparados para hacer esta cuenta, te dejo a ti que repliques la cuenta si quieres con esta fórmula. Yo lo intenté previamente y me salió -0.44 (hay que entender que la similitud coseno puede ir de -1 a 1 con esta fórmula). La interpretación que se da a esto es que, por una parte los usuarios 1 y 11 tienen un 0.97 de similitud, lo que significa que han puntuado de manera similar las películas que tienen en común (como se ve arriba), sin embargo, tienen baremos diferentes para las películas que han valorado en la plataforma, esto se refleja en la diferencia que encontramos si restamos las medias.

# Métricas

La segunda cuestión va relacionada con las métricas asociadas al recomendador. La primera de todas, vamos a mirar el RMSE, que en este caso sería la predicción del rating que un usuario haría a una película dada.

In [26]:
model.test(testset)

[Prediction(uid=140, iid=6765, r_ui=3.5, est=3.2944185355225772, details={'actual_k': 7, 'was_impossible': False}),
 Prediction(uid=603, iid=290, r_ui=4.0, est=4.278240643256959, details={'actual_k': 9, 'was_impossible': False}),
 Prediction(uid=438, iid=5055, r_ui=4.0, est=2.497394210889849, details={'actual_k': 5, 'was_impossible': False}),
 Prediction(uid=433, iid=164179, r_ui=5.0, est=3.929369553823387, details={'actual_k': 21, 'was_impossible': False}),
 Prediction(uid=474, iid=5114, r_ui=4.0, est=3.503229285466356, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=304, iid=1035, r_ui=4.0, est=3.9645214632385155, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=298, iid=4974, r_ui=1.0, est=3.0786221295696077, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid=131, iid=293, r_ui=4.0, est=4.176201992756822, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=288, iid=5784, r_ui=2.5, est=2.7

In [17]:
accuracy.rmse(model.test(testset))

RMSE: 0.9823


0.9822558142846856

In [18]:
# De la misma manera puedo usar el MAE
accuracy.mae(model.test(testset))

MAE:  0.7559


0.7558972904135215

La primera métrica que tenemos es que el recomendador estima con un error de 0.98 puntos de RMSE y 0.75 puntos de media. Intuitivamente esto quiere decir que, si a un usuario le recomiendo una película con una puntuación estimada de 3.5, la puntuación que espero que el usuario me devuelva, debe encontrarse entre 2.5 y 4.5 aproximadamente. 

Por otro lado, revisa el código del curso (RecommenderMetrics.py) para que puedas ver algunas métricas que ya se han programado y las puedas ir usando en este caso. Luego las tendremos que implementar para recomendadores más difíciles. Para empezar esas métricas lo primero es sacar las mejores recomendaciones. Se puede hacer usando la librería Surprise o el código del curso que has ido viendo.

In [28]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [22]:
antitest = trainset.build_anti_testset()
antitest_pred = model.test(antitest)

In [32]:
antitest_pred

[Prediction(uid=432, iid=474, r_ui=3.503229285466356, est=3.7632797496617276, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=432, iid=4351, r_ui=3.503229285466356, est=3.168283291883526, details={'actual_k': 12, 'was_impossible': False}),
 Prediction(uid=432, iid=2987, r_ui=3.503229285466356, est=3.701803417319988, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=432, iid=177, r_ui=3.503229285466356, est=2.4905428014309874, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid=432, iid=750, r_ui=3.503229285466356, est=4.3888541766407965, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=432, iid=6503, r_ui=3.503229285466356, est=2.3463220259675346, details={'actual_k': 23, 'was_impossible': False}),
 Prediction(uid=432, iid=8641, r_ui=3.503229285466356, est=3.8027177324023103, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=432, iid=1203, r_ui=3.503229285466356, est=4.362866342029792, details={'a

In [30]:
topN = get_top_n(antitest_pred, n=10)

In [31]:
topN

defaultdict(list,
            {432: [(4135, 5.0),
              (2196, 5.0),
              (6122, 5.0),
              (115727, 5.0),
              (107951, 5.0),
              (4956, 5.0),
              (2972, 5.0),
              (124404, 5.0),
              (2007, 5.0),
              (149350, 5.0)],
             288: [(4135, 5.0),
              (2196, 5.0),
              (115727, 5.0),
              (107951, 5.0),
              (4956, 5.0),
              (2972, 5.0),
              (124404, 5.0),
              (2007, 5.0),
              (149350, 5.0),
              (5537, 5.0)],
             599: [(4135, 5.0),
              (2196, 5.0),
              (6122, 5.0),
              (115727, 5.0),
              (107951, 5.0),
              (4956, 5.0),
              (2972, 5.0),
              (124404, 5.0),
              (2007, 5.0),
              (149350, 5.0)],
             42: [(4135, 5.0),
              (2196, 5.0),
              (6122, 5.0),
              (115727, 5.0),
              (1

In [36]:
# Me he fijado que se repiten los mismos numeros, voy a echarles un vistazo
ratings[ratings["movieId"] == 4135]

,userId,movieId,rating,timestamp
87219,562,4135,5.0,1368896069


In [37]:
ratings[ratings["movieId"] == 2196]

,userId,movieId,rating,timestamp
91608,594,2196,5.0,1108975245


Al final la conclusión es que tener similitudes tan altas si una pelicula solamente es valorada una vez con un 5, todas las combinaciones entre usuarios donde haya algo de similitud (es decir, casi todos) van a recibir esta recomendación. Este efecto se podría evitar haciendo una matriz de similitudes "bien hecha". 

Iba a continuar con algún ejemplo de métrica, pero viendo las recomendaciones, no le veo mucho sentido, sería mejor cambiar el algoritmo o construirlo de otra manera y revisar las métricas con más detalle.

Te dejo unos ejemplos de Surprise con código para que puedas echarle un ojo: https://github.com/NicolasHug/Surprise/tree/c1de6b0e35726577e0625b2ed77688655610ad54/examples